# P2WSH MultiSig BIP322
## Role: Signer

## 1. Generate HD Key from mneumonic

In [1]:
from buidl.mnemonic import secure_mnemonic
from buidl.hd import HDPrivateKey

# mnemonic = secure_mnemonic()

mnemonic = 'army injury wrestle spin loyal skill acid word hobby prosper purity disorder impulse observe elevator perfect deer student juice reveal portion nice quarter banana'

hd_root = HDPrivateKey.from_mnemonic(mnemonic)
root_xfp = hd_root.fingerprint().hex()

## 2. Select Key to Sign With

In [2]:
bip32_path = "m/45'/0/0/11"

signing_privkey = hd_root.traverse(bip32_path)
signing_pubkey = signing_privkey.pub.point


## 3. Share Public Key with Creator

The public key to include in a p2sh redeem script must be communicated to the creator of a BIP322 signature through some communication channel. This channel is out of scope.

In this example we will copy a hex representation of the public key across to the [PSBT Creator notebook](http://localhost:8888/notebooks/p2sh_multisig_bip322/PSBT_creator.ipynb)

In [4]:
print("Copy the below line into step 3 of the creator notebook \n")
print(f"signer_2_pubkey = '{signing_pubkey.sec().hex()}'")

Copy the below line into step 3 of the creator notebook 

signer_2_pubkey = '035ec5bae25dfb8c06ba3f996d5c44b056a94ca00ec3285fc3daa9721e7624ac5b'


## 4. Recieve PSBT from Creator

Some out of band communication mechanism

In [5]:
#### Copy from PSBT_Creator notebook Step 8. ######

b64_psbt = 'cHNidP8BAD0AAAAAAeTHmyyL0BtEMKgIYJ4VxLouFJf/k9NLF6+gu5yMrmP7AAAAAAAAAAAAAQAAAAAAAAAAAWoAAAAAAAEBKwAAAAAAAAAAIgAg+BfBB30QVG3ji/uY8DEIazOzs7e2IxjyGSVRC3EHnxcBBWlTIQP3dyWvTZjUENWJowMWBsQrrXCUs20Gu5YF79CG5Ga0XSEDwqI5GVBOuFkFzQOGH5eTExSAj2Z/LDV/hbcvAPQdlJMhA17FuuJd+4wGuj+ZbVxEsFapTKAOwyhfw9qpch52JKxbU64iBgP3dyWvTZjUENWJowMWBsQrrXCUs20Gu5YF79CG5Ga0XRR7BeobLQAAgAAAAAAAAAAABgAAAAAA'


In [6]:
from buidl.psbt import PSBT
psbt_to_sign = PSBT.parse_base64(b64_psbt)

In [11]:
psbt_to_sign.psbt_ins[0].prev_tx

## 5. TODO: How does the signer Verify the Message that they are signing with this

I assume they must be given the means to construct the to_spend transaction themselves and verify that this is referenced as the only input to the tx_obj in the PSBT

## 6. Update PSBT with pubkey

In [7]:
from buidl.psbt import NamedHDPublicKey

pubkey_lookup = {}

named_hd_pubkey_obj = NamedHDPublicKey.from_hd_pub(
    child_hd_pub=signing_privkey.pub,
    xfp_hex=root_xfp,
    path=bip32_path,
)
# pubkey lookups needed for validation
pubkey_lookup[named_hd_pubkey_obj.sec()] = named_hd_pubkey_obj

In [13]:
# Note tx_lookup is set to None, because already been defined by creator
psbt_to_sign.update({}, pubkey_lookup)

## 7. Sign the PSBT

In [14]:
psbt_to_sign.sign(hd_root)

True

## 9. Serialize Signed PSBT and Return to Creator

Note: Communication channel out of band. We use copy and paste.

In [15]:
signer_2_psbt_b64 = psbt_to_sign.serialize_base64()
print("Copy this VVVV \n\n")
print(f"signer_2_psbt_b64 = '{signer_2_psbt_b64}'")

Copy this VVVV 


signer_2_psbt_b64 = 'cHNidP8BAD0AAAAAAeTHmyyL0BtEMKgIYJ4VxLouFJf/k9NLF6+gu5yMrmP7AAAAAAAAAAAAAQAAAAAAAAAAAWoAAAAAAAEBKwAAAAAAAAAAIgAg+BfBB30QVG3ji/uY8DEIazOzs7e2IxjyGSVRC3EHnxciAgNexbriXfuMBro/mW1cRLBWqUygDsMoX8PaqXIediSsW0gwRQIhAIMbaYnbFNGeLv/uUQfXMnkLaAluTXK3iyZll2fJFwmTAiAUdbF6BIp4WROAhfT/UkxiHDLwrmY2OheRdF01S4/UowEBBWlTIQP3dyWvTZjUENWJowMWBsQrrXCUs20Gu5YF79CG5Ga0XSEDwqI5GVBOuFkFzQOGH5eTExSAj2Z/LDV/hbcvAPQdlJMhA17FuuJd+4wGuj+ZbVxEsFapTKAOwyhfw9qpch52JKxbU64iBgNexbriXfuMBro/mW1cRLBWqUygDsMoX8PaqXIediSsWxQAHQ2eLQAAgAAAAAAAAAAACwAAACIGA/d3Ja9NmNQQ1YmjAxYGxCutcJSzbQa7lgXv0IbkZrRdFHsF6hstAACAAAAAAAAAAAAGAAAAAAA='
